# Streaming

In [ ]:
## Create LLM
from langchain_ollama import ChatOllama

llm = ChatOllama(model="deepseek-r1:1.5b", temperature=0.6)

In [ ]:
## Create memory for LLM
from langgraph.checkpoint.memory import MemorySaver

memory=MemorySaver()

In [ ]:
## Create State
from typing import Annotated
from typing_extensions import TypedDict

class State(TypedDict):
    messages:Annotated

## Create Node
def superbot(state:State):
    return {"messages": [llm.invoke(state["messages"])]}

In [ ]:
## Create the Graph
from langgraph.graph import StateGraph, START, END

graph_builder=StateGraph(State)

## Adding Node
graph_builder.add_node("superbot",superbot)

## Adding Edges
graph_builder.add_edge(START, "superbot")
graph_builder.add_edge("superbot", END)

## Compile the graph
graph=graph_builder.compile(checkpointer=memory)
graph

In [ ]:
## Create a Thread
config={"configurable":{"thread_id":"101"}}

## Invoke the graph 
for chunk in graph.stream({"messages":"Hi, My name is Rahul and I love to play cricket"},config=config,stream_mode="updates"):
    print(chunk)

In [ ]:
## Invoke the graph 
for chunk in graph.stream({"messages":"I also love play batminton"},config=config,stream_mode="values"):
    print(chunk)

In [ ]:
## Create a Thread
config={"configurable":{"thread_id":"2649"}}

## Invoke the graph 
async for chunk in graph.astream_events({"messages":"Hi, My name is Rahul and I love to play cricket"},config=config,version="v2"):
    print(chunk)